In [1]:
import sys
import nivapy
import qgrid
import pandas as pd

# Quick hack to import from relative folder
sys.path.insert(0, '../ndbview')
from ndbview import ndb_queries

In [2]:
# Connect to db
ora_eng = nivapy.da.connect()

Connection successful.


In [3]:
# List projects
prj_df = ndb_queries.get_all_projects(ora_eng)

# Convert to qgrid
prj_grid = qgrid.QgridWidget(df=prj_df, show_toolbar=False)
prj_grid

QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [4]:
# Show selected projects
sel_prj = prj_grid.get_selected_df()
sel_prj

,project_id,o_number,project_name,project_description
4,94,25186,VIV,Data til Vestfold Interkommunale vannverk
5,95,26069,Glomma,Visning av Glomma data
6,96,12019 08,Borrevannet,Overvåking av Borrevannet og Adalstjern i Hort...
7,96,11025 03,Borrevannet,Overvåking av Borrevannet og Adalstjern i Hort...
3,91,14052,Oslofjord,Målinger i indre Oslofjord


In [5]:
# Get stations
stn_df = ndb_queries.get_project_stations(sel_prj, ora_eng)

# Convert to qgrid
stn_grid = qgrid.QgridWidget(df=stn_df, show_toolbar=False)

print len(stn_df)
stn_grid

40


QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [6]:
# Get parameters
par_df = ndb_queries.get_station_parameters(stn_df, ora_eng)

# Convert to qgrid
par_grid = qgrid.QgridWidget(df=par_df, show_toolbar=False)
par_grid

QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [7]:
# Show selected parameters
sel_par = par_grid.get_selected_df()
sel_par

,parameter_id,parameter_name,unit
5,221,Cd,µg/l
6,6,Cl,mg/L
7,382,CODMn,mg O/l
8,629,Cr,µg/l
9,610,Cu,µg/L
10,492,Cu,mg/l
4,1,Ca,mg/L


In [8]:
# User options
st_dt = '1990-01-01'
end_dt = '2017-12-31'
lod_flags = False

# Get water chemsitry
wc_df, dup_df = ndb_queries.get_chemistry_values(stn_df, sel_par, 
                                                 st_dt, end_dt, 
                                                 lod_flags, ora_eng)

print len(dup_df)
print len(wc_df)

    WARNING
    The database contains duplicated values for some station-date-parameter combinations.
    Only the most recent values will be used, but you should check the repeated values are not errors.
    The duplicated entries are returned in a separate dataframe.

4
140


In [9]:
# Convert duplicates to qgrid
dup_grid = qgrid.QgridWidget(df=dup_df, show_toolbar=False)
dup_grid

QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)

In [10]:
# Convert chem to qgrid
wc_grid = qgrid.QgridWidget(df=wc_df, show_toolbar=False)
wc_grid

QgridWidget(grid_options={'highlightSelectedRow': True, 'fullWidthRows': True, 'rowHeight': 28, 'enableColumnReorder': False, 'filterable': True, 'sortable': True, 'enableTextSelectionOnCells': True, 'editable': True, 'explicitInitialization': True, 'minVisibleRows': 8, 'autoEdit': False, 'syncColumnCellResize': True, 'maxVisibleRows': 15, 'defaultColumnWidth': 150, 'forceFitColumns': True, 'highlightSelectedCell': False}, precision=5)